Here's how you can load file from your google Drive. (the path might change)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import gzip

with gzip.open('/content/gdrive/My Drive/Columbia/Personalization/Final Project/Beeradvocate.txt.gz', 'r') as f:
  rb_file = f.readlines()

In [3]:
#Use this if your file is in txt format

with open('your path', 'r') as f:
  rb_file = f.readlines()

FileNotFoundError: ignored

In [3]:
import gzip

with gzip.open('Beeradvocate.txt.gz', 'r') as f:
  rb_file = f.readlines()

In [6]:
# python2
import pandas as pd
import numpy as np
import os

In [4]:
data = []
row_out = []

for i in rb_file:
    row = i.decode('utf-8', errors = 'ignore')
    #print(row)
    if row == '\n':
      data.append(row_out)
      row_out = []
      continue
    row = row.split(":", 1)[1].rstrip()
    row_out.append(row)

        
        

In [7]:
data = pd.DataFrame(data)

data.columns = ['beer_name', 'beer_beerId', 'beer_brewer', 'beer_ABV', 'beer_style', 
                'review_appearance', 'review_aroma', 'review_palate', 'review_taste', 
                'review_overall', 'review_time', 'review_profileName', 'review_text']

# keep 3 columns: user name, beer name, overall score
data2 = data[['beer_name', 'review_profileName', 'review_overall','review_time']]
#print(data2.shape)

# remove NA
data2 = data2[pd.notnull(data2.beer_name)]
data2 = data2[pd.notnull(data2.review_profileName)]
data2 = data2[pd.notnull(data2.review_overall)]
#print (data2.shape)
#data2.head(10)

In [8]:
# keep top 1000/33382 frequent users
user = data2.review_profileName.value_counts()
user_list = user.keys()[:1000].tolist()

# keep top 100/56855 most reviewed beer
beer = data2.beer_name.value_counts()[:100]
beer_list = beer.keys()[:100].tolist()

# keep (beer&user) pair in (user_list) and (beer_list)
subdata = data2[data2.beer_name.isin(beer_list)]
subdata = subdata[subdata.review_profileName.isin(user_list)]

# sort by user names
subdata = subdata.sort_values(by=['review_profileName','beer_name','review_time'])

print(subdata.shape)
subdata.head(10)

(68929, 4)


,beer_name,review_profileName,review_overall,review_time
1191634,120 Minute IPA,1fastz28,3,1198733748
1170777,60 Minute IPA,1fastz28,4,1176428202
1174390,90 Minute IPA,1fastz28,4.5,1173146934
1339659,Alpha King Pale Ale,1fastz28,4.5,1275961162
868119,Arrogant Bastard Ale,1fastz28,4,1214535830
1032119,Ayinger Celebrator Doppelbock,1fastz28,4,1240105718
931404,Bell's Hopslam Ale,1fastz28,4.5,1236131835
944912,Bell's Oberon Ale,1fastz28,4.5,1240099321
353066,Blue Moon Belgian White,1fastz28,3,1204429269
591597,Bourbon County Brand Stout,1fastz28,4,1207029127


Only keeping latest reviews by user in case they rated the same beer multipe times

In [9]:
#subdata.groupby(['beer_name', 'review_profileName'])['review_time'].max()

data_clean = subdata.sort_values(by='review_time')

data_clean.drop_duplicates(['beer_name', 'review_profileName'], keep = 'last',
                          inplace = True)


In [10]:
data_clean.head(10)
#data_clean.shape

,beer_name,review_profileName,review_overall,review_time
282223,Samuel Adams Boston Lager,Jason,4,1000083887
518392,Samuel Smith's Oatmeal Stout,Jason,4,1000336190
518391,Samuel Smith's Oatmeal Stout,guinness33,5,1000464943
1339361,Alpha King Pale Ale,John,5,1000733852
1541504,Golden Monkey,John,5,1000809985
1334473,Dreadnaught IPA,John,5,1000890164
1544699,HopDevil Ale,John,5,1000904483
91268,Sierra Nevada Pale Ale,John,4,1000905152
518390,Samuel Smith's Oatmeal Stout,stoutman,5,1000931285
1548897,Storm King Stout,John,5,1001336489


**Splitting the data into training and testing**

In [11]:
data_clean['review_overall'] = data_clean['review_overall'].astype(float)

data_train = data_clean.sample(frac = 0.7)
data_test = data_clean.drop(data_train.index)

Calculate the average rating for each item

In [12]:
mean_rating = data_train.groupby(['beer_name'], as_index = False, sort = False)['review_overall'].mean().rename(columns = {'review_overall': 'item_mean'})

data_train = pd.merge(data_train, mean_rating, on = 'beer_name', how = 'left')
data_train['adjusted_rating'] = data_train['review_overall'] - data_train['item_mean']

In [13]:
data_train.head()

,beer_name,review_profileName,review_overall,review_time,item_mean,adjusted_rating
0,Hop Wallop,Wasatch,4.5,1133483572,4.013771,0.486229
1,Alpha King Pale Ale,OWSLEY069,4.5,1220111363,4.319231,0.180769
2,Young's Double Chocolate Stout,erosier,4.0,1202677755,4.062637,-0.062637
3,Hoegaarden Original White Ale,bjohnson,4.0,1153266913,4.063584,-0.063584
4,Pabst Blue Ribbon (PBR),KTCamm,4.5,1234141880,3.436860,1.063140


**Build the similarity matrix**

In [13]:
def item_matrix(train_sample):
  #find distinct beers
  beers = np.unique(train_sample['beer_name'])
  beer_pairs = []

  for beer1 in beers:
    #determine users who consumed the beer
    users = train_sample.loc[train_sample['beer_name'] == beer1] 
    for index, user in users.iterrows():
      #find rating for first beer
      user_name = user['review_profileName']  
      beer1_rating = user['adjusted_rating']
      #find other beer ratings by same users, excluding beer1 rating
      user_ratings = train_sample.loc[(train_sample['review_profileName'] == user_name) & (train_sample['beer_name'] != beer1)]
      for index, beer2 in user_ratings.iterrows():
        beer2_name = beer2['beer_name']
        beer2_rating = beer2['adjusted_rating']
        #print([user_name, beer1, beer2_name, beer1_rating, beer2_rating])
        beer_pairs.append([user_name, beer1, beer2_name, beer1_rating, beer2_rating])
  
  beer_pairs = pd.DataFrame(beer_pairs, columns = ['user', 'beer1', 'beer2', 'scaled_r1', 'scaled_r2'])
  
  return beer_pairs.groupby(['beer1', 'beer2'])[['scaled_r1','scaled_r2']].corr(method = 'pearson')
      

In [14]:
beer_pairs = item_matrix(data_train)

In [36]:
print (beer_pairs.shape)
beer_corr = beer_pairs.iloc[0::2][['scaled_r2']].reset_index()[['beer1', 'beer2', 'scaled_r2']]
beer_corr = beer_corr.rename(columns = {'scaled_r2': 'Pearson_corr'})

beer_corr.to_csv('beer_corr.csv', index = False, encoding = 'utf-8')

(19800, 2)


In [21]:
beer_corr = pd.read_csv('beer_corr.csv', encoding = 'utf-8')
beer_corr = beer_corr.rename(columns = {'scaled_r2': 'Pearson_corr'})
beer_corr.head()

,beer1,beer2,Pearson_corr
0,#9,120 Minute IPA,0.003980
1,#9,60 Minute IPA,0.074866
2,#9,90 Minute IPA,0.052636
3,#9,Alpha King Pale Ale,-0.011340
4,#9,Anchor Steam Beer,0.029190


In [42]:
def predict(user, beer, n_neighbors):
    #find the beers that have been rated by the user 
    user_ratings = data_train.loc[(data_train['review_profileName'] == user) & (data_train['beer_name'] != beer)]
    user_ratings = user_ratings.rename(columns = {'beer_name': 'beer2'})
    #find list of potential neighbors
    similarities = beer_corr.loc[beer_corr['beer1'] == beer]
    #narrow down to neighbors that were rated by users
    neighbors = user_ratings.merge(similarities, how = 'inner', on='beer2')
    #select n nearest neighbors
    neighbors = neighbors.sort_values(by = ['Pearson_corr'], ascending = False)[0:n_neighbors]
    #calculate numerator
    num = sum(neighbors['Pearson_corr']*neighbors['adjusted_rating'])
    den = sum(abs(neighbors['Pearson_corr']))
    #find item mean
    beer_mean = data_train.loc[data_train['beer_name'] == beer]['item_mean'][0]
    return beer_mean + num/den

In [44]:
t_u = data_train['review_profileName'][0]
t_b = data_train['beer_name'][0]
predict(t_u, t_b, 5)

4.4263192281914705

In [25]:
beer_corr.loc[beer_corr['beer1'] == ' 60 Minute IPA'].sort_values(by = ['Pearson_corr'], ascending = False)[0:10]

,beer1,beer2,Pearson_corr
200,60 Minute IPA,90 Minute IPA,0.284617
294,60 Minute IPA,Yeti Imperial Stout,0.223310
290,60 Minute IPA,Two Hearted Ale,0.221110
286,60 Minute IPA,Trappistes Rochefort 10,0.220140
293,60 Minute IPA,World Wide Stout,0.217669
289,60 Minute IPA,Tröegs Nugget Nectar,0.203313
222,60 Minute IPA,Dreadnaught IPA,0.201900
267,60 Minute IPA,Samuel Adams Winter Lager,0.200865
261,60 Minute IPA,Racer 5 India Pale Ale,0.198971
213,60 Minute IPA,Burton Baton,0.196168
